# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121487e+02     1.504808e+00
 * time: 0.08546805381774902
     1     1.084284e+01     9.081558e-01
 * time: 0.8925919532775879
     2    -1.089684e+01     1.094589e+00
 * time: 1.001072883605957
     3    -3.368320e+01     8.076531e-01
 * time: 1.1591110229492188
     4    -4.706680e+01     6.045808e-01
 * time: 1.304063081741333
     5    -5.676613e+01     2.274354e-01
 * time: 1.4467508792877197
     6    -5.978214e+01     1.281292e-01
 * time: 1.5507218837738037
     7    -6.090392e+01     5.066956e-02
 * time: 1.6664729118347168
     8    -6.131397e+01     7.005384e-02
 * time: 1.7914528846740723
     9    -6.160558e+01     3.561403e-02
 * time: 1.896190881729126
    10    -6.182144e+01     3.145040e-02
 * time: 2.007904052734375
    11    -6.198791e+01     2.031175e-02
 * time: 2.118726968765259
    12    -6.205370e+01     1.635385e-02
 * time: 2.222411870956421
    13    -6.211798e+01     1.361255e-02
 * time: 2.333299875259399

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557672
    AtomicNonlocal      14.8522634
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336824

    total               -62.261666460804
